In [64]:
import numpy as np


def eclidean_distance(point1, point2):
    return np.sqrt(np.sum(np.power(point1-point2, 2)))
def mse(point1, point2):
    return np.sum((point1, point2)) ** 2

def distance(cluster1,cluster2,type_=mse):
    return type_(cluster1, cluster2)

def min_distance(arr1, arr2, X):
    min_dist = float('inf')
    
    for idx1 in arr1:
        for idx2 in arr2:
            dist = np.linalg.norm(X[idx1] - X[idx2])
            if dist < min_dist:
                min_dist = dist
    
    return min_dist

def distance_mat(clusters, data):
    cluster_len = len(clusters)
    dist_mat = np.zeros((cluster_len, cluster_len))
    for i1 in range(cluster_len):
        for i2 in range(cluster_len):
            if i2 > i1:
                dist_mat[i1, i2] = min_distance(clusters[i1], clusters[i2], data)
            else : 
                dist_mat[i1, i2] = np.inf
    return dist_mat

def min_matrix(matrix):
    min_index = np.argmin(matrix)
    _, cols = matrix.shape
    return min_index // cols, min_index % cols
    
def linkage(clusters, data, num_clusters):
    for i in range(num_clusters):
        print(clusters)
        dist_mat = distance_mat(clusters, data)
        i_min, j_min = min_matrix(dist_mat)
        clusters[i_min] = clusters[i_min] + clusters[j_min]
        clusters.pop(j_min)
        print(dist_mat)

In [57]:
from sklearn.datasets import make_blobs

num_clusters = 1
num_samples = 3
X, y = make_blobs(n_samples=num_samples, centers=num_clusters, cluster_std=2, random_state=42)
X += 10

In [67]:
X = [
    [4, 4, 1, 1],
    [3, 0, 1, 1],
    [8, 2, 1, 1],
    [8, 0, 1, 1],
    [8.5, 0, .5, .5]
]
clusters = [
    [0], [1], [2], [3], [4]
]
linkage(clusters, np.asarray(X), 5)

[[0], [1], [2], [3], [4]]
[[       inf 4.12310563 4.47213595 5.65685425 6.06217783]
 [       inf        inf 5.38516481 5.         5.54526825]
 [       inf        inf        inf 2.         2.17944947]
 [       inf        inf        inf        inf 0.8660254 ]
 [       inf        inf        inf        inf        inf]]
[[0], [1], [2], [3, 4]]
[[       inf 4.12310563 4.47213595 5.65685425]
 [       inf        inf 5.38516481 5.        ]
 [       inf        inf        inf 2.        ]
 [       inf        inf        inf        inf]]
[[0], [1], [2, 3, 4]]
[[       inf 4.12310563 4.47213595]
 [       inf        inf 5.        ]
 [       inf        inf        inf]]
[[0, 1], [2, 3, 4]]
[[       inf 4.47213595]
 [       inf        inf]]
[[0, 1, 2, 3, 4]]
[[inf]]
